# [Promises for asynchronous programming](https://exploringjs.com/impatient-js/ch_promises.html)

## The basics of using Promises

* promises are a technique for delivering results asynchronously

### Using a Promise-based function

* Promises are similar to the event pattern
    - there is an object (Promise) where we register callbacks:
        * Method .then() registers callbacks that handles results
        * Method .catch() registers callbacks that handle errors
* a Promise-based function returns a Promise and sends it a result or an error (if and when it is done)
    - the promise passes it on to the relevant callbacks
* in contrast to the event pattern, Promises are optimized for one-off results:
    - a result (or an error) is cached so that it doesn't matter if we register a callback before or after the result (or error) was sent
    - .then() and .catch() can both be chained b/c they return Promises
        * helps with sequentially invoking multiple asynchronous functions

In [3]:
Promise-based function
addAsync(3, 4)
    .then(result => { // success
        assert.equal(result, 7);
    })
    .catch(error => { // failure
        assert.fail(error);
    })

const addPositiveAsync = (x, y) => {
    return new Promise((resolve, reject) => {
      if (x < 0 || y < 0) {
        reject('X and Y should be positive numbers!');
    }
    else {
        setTimeout(() => {
          resolve(x + y);
      }, 1000)
    }
  });
}

const callAsync = async () => {
    try {
        let result = await addPositiveAsync(-1, -2);
    console.log(result);
  }
  catch(error) {
      console.error(error);
  }
  
  try {
      let result = await addPositiveAsync(1, 2);
        console.log(result);
  }
  catch(error) {
      console.error(error);
  }
}

callAsync();

Promise { <pending> }

X and Y should be positive numbers!


3


### What is a Promise?

* two ways of looking at it:
    1. placeholder or container for the final result that will eventually be delivered
    2. an object with which we can register listeners

### Implementing a Promise-based function

* addAsync() immediately invokes the Promise constructor
* the implementation of the function resides in the callback that is passed to the constructor (line A)
* the callback is provided with 2 functions:
    1. resolve is used for delivering a result (in case of success)
    2. reject is used for delivering an error (in case of failure)

In [ ]:
// implementation of a Promise-based function that adds two numbers x and y:
function addAsync(x, y) {
    return new Promise(
        (resolve, reject) => { // A
            if (x === undefined || y === undefined) {
                reject(new Error('Must provide two parameters'));
            }
            else {
                resolve(x + y);
            }
        }
    )
}

### States of Promises

![States of Promises](https://exploringjs.com/impatient-js/img-book/promises/promise_states_simple.svg)

* a Promise can be in one of three states:
    1. pending
    2. fulfilled
    3. rejected
* if a Promise is in a final (non-pending) state, it is called settled
* promises specialize in one-off results and protect us against race conditions
    - race conditions = registering too early or too late
    - if we register .then() or .catch() callback too early, it is notified once a Promise is settled
    - once a Promise is settled, the settlement value (result or error) is cached
        * so .then() or .catch() are called after the settlement, they receive the cached value
* once a Promise is settled, its state and settlemenet value can't change anymore
    - this helps make the code predictable and enforces the one-off nature of Promises

### Promise.resolve(): create a Promise fulfilled with a given value

In [ ]:
Promise.resolve(123)
    .then(x => {
        assert.equal(x, 123);
    })

In [ ]:
// if the parameter is already a Promise, it is returned unchanged
// so, given ab arbitrary value x, we can use Promise.resolve(x) to ensure we have a Promise
// the name is resolve, and not fulfill, b/c .resolve() returns a rejected Promise if its Parameter is a rejected Promise

const abcPromise = Promise.resolve('abc');
assert.equal(
    Promise.resolve(abcPromise),
    abcPromise
)

### Promise.reject() create a Promise rejected with a given value

In [ ]:
const myError = new Error('My error!');
Promise.reject(myError)
    .catch(err => {
        assert.equal(err, myError);
    })

### Returning and throwing in .then() callbacks

* .then() handles Promise fulfillments
* it also returns a fresh Promise
* how that Promise is settled depends on what happens inside the callback and there are three common cases

#### Returning a non-Promise value

* first, the callback can return a non-Promise value (line A)
* the Promise returned by .then() is fulfilled with that value (in line B)

In [ ]:
Promise.resolve('abc')
    .then(str => {
        return str + str; // A
    })
    .then(str2 => {
        assert.equal(str2, 'abcabc'); // B
    })

#### Returning a Promise

* second, the callback can return a Promise p (line A)
* consequently, p "becomes" what .then() returns
    - i.e. the Promise that .then() has already returned is effectively replaced by p
* why is that useful?
    - b/c we can return the result of a Promise-based operation and process its fulfillment value via a "flat" (non-nested) .then()
    - basically, we can just chain .then() methods instead of having them nested inside of 1

In [ ]:
Promise.resolve('abc')
    .then(str => {
        return Promise.resolve(123); // A
    })
    .then(num => {
        assert.equal(num, 123)
    })

In [ ]:
// Flat
asyncFunc1()
    .then(result1 => {
        /*...*/
        return asyncFunc2();
    })
    .then(result2 => {
        /*...*/
    });

// Nested
asyncFunc1()
    .then(result1 => {
        /*...*/
        asyncFunc2()
            .then(result2 => {
                /*...*/
            })
    })

#### Throwing an exception

* third, the callback can throw an exception
* the Promise returned by .then() is rejected with that exception
    - that is, a synchronous error is converted into an asynchronous error


In [ ]:
const myError = new Error('My error!');
Promise.resolve('abc')
    .then(str => {
        throw myError;
    })
    .catch(err => {
        assert.equal(err, myError);
    })

### .catch() and its callback

* the difference between .then() and .catch() is that the latter is triggered by rejections, not fulfillments
    - but both methods rurn the actions of their callbacks into Promises in the same manner
* for example, the value returned by the .catch() callback in line A becomes a fulfillment value

In [ ]:
const err = new Error();

Promise.reject(err)
    .catch(e => {
        assert.equal(e, err);
        // something went wrong, use a default value
        return 'default value'; // A
    })
    .then(str => {
        assert.equal(str, 'default value')
    })

### Chaining method calls

* .then() and .catch() __always return Promises__
* this allows us to create arbitrary long chains of method calls
* due to chaining, the return in line A returns the result of the last .then()
* in a way, .then() is the asynchronous version of the synchronous semicolon
    - .then() executes 2 asynchronous operations sequentially
    - the semicolon executes 2 synchronous operations sequentially

In [ ]:
function myAsyncFunc() {
    return asyncFunc1() // A
        .then(result1 => {
            // ...
            return asyncFunc2(); // a Promise
        })
        .then(result2 => {
            // ...
            return result2 ?? '(Empty)'; // not a Promise
        })
        .then(result3 => {
            // ...
            return asyncFunc4(); // a Promise
        })
}

* we can also add .catch() into the mix and let it handle multiple error sources at the same time

In [ ]:
asyncFunc1()
    .then(result1 => {
        // ...
        return asyncFunction2();
    })
    .then(result2 => {
        // ...
    })
    .catch(error => {
        // Failure: handle errors of asyncFunc1(), asyncFunc2()
        // and any (sync) exceptions thrown in previous callbacks
    })

### .finally()

* the .finally() callback is always executed - independently of somePromise and the values returned by .then() and/or .catch()
* in contrast:
    - the .then() callback is only executed if somePromise is fulfilled
    - the .catch() callback is only executed if:
        * either somePromise is rejected
        * or the .then() callback returns a rejected Promise
        * or the .then() callback throws an exception

In [ ]:
somePromise
    .then((result) => {
        // ...
    })
    .catch((error) => {
        // ...
    })
    .finally(() => {
        // ...
    });

* .finally() ignores what its callback returns and simply passes on the settlement that existed before it was called:

In [ ]:
Promise.resolve(123)
    .finally(() => {})
    .then((result) => {
        assert.equal(result, 123);
    })

Promise.reject('error')
    .finally(() => {})
    .catch((error) => {
        assert.equal(error, 'error')
    })

* if the .finally() callback throws an exception, the Promise returned by .finally() is rejected

In [ ]:
Promise.reject('error (originally)')
    .finally(() => {
        throw 'error (finally)';
    })
    .catch((error) => {
        assert.equal(error, 'error (finally)');
    })

##### Use case for .finally(): cleaning up

* cleaning up after you are done with a resource regardless if it was successful or failed

In [ ]:
let connection;

db.open()
.then((conn) => {
    connection = conn;
    return connection.select({ name: 'Jane '})
})
.then((result) => {
    // Process result
    // Use 'connection' to make more queries
})
//...
.catch((error) => {
    //handle errors
})
.finally(() => {
    connection.close();
});

##### Use case for .finally(): doing something first after any kind of settlement

* can use .finally() before both .then() and .catch()
* .finally() will be executed before the other two callbacks

In [2]:
Promise.resolve('fulfilled')
    .finally(() => {
        console.log('finally');
    })
    .then((result) => {
        console.log('then ' + result);
    })
    .catch((error) => {
        console.log('catch ' + error);
    })

// Output:
// 'finally'
// 'then fulfilled'

Promise.reject('rejected')
    .finally(() => {
        console.log('finally');
    })
    .then((result) => {
        console.log('then ' + result);
    })
    .catch((error) => {
        console.log('catch ' + error);
    })

// Output:
// 'finally'
// 'catch rejected'

Promise { <pending> }

finally
finally
then fulfilled
catch rejected


### Advantages of promises over plain callbacks

* when it comes to handling one-off results:
    - parameters in Promise-based functions and methods are cleaner
        * in a Promise, you only have resolve and reject to worry about as far as parameters go
        * with callbacks, the first couple of arguments are for input to the function while the last arguments are for the actual callback function
            - so determining what is an input parameter and what is a callback function can be confusing
    - chaining asynchronous processing steps is more convenient
    - Promises handle both asynchronous errors (via rejections) and synchronous errors
        * inside the callbacks for new Promise(), .then(), and .catch(), exceptions are converted to rejections
        * if we use callbacks, exceptions are not handled for us
    - Promises are a single standard
* also the biggest advantage of Promises is being able to use the async/await syntax
    - async/await is a synchronous-looking syntax for performing asynchronous computations

## Error handling: don't mix rejections and exceptions

* don't mix (asynchronous) rejections and (synchronous) exceptions
* simplifies our code b/c we only need to focus on one error-handling mechanic
* so for Promise-based functions/methods, they should never throw exceptions

In [ ]:
// DON'T DO THIS:

function asyncFunc() {
    doSomethingSync(); // A
    return doSomethingAsync()
        .then(result => {
        // ...
    })
}

* if doSomethingSync in line A throws an exception, then asyncFunc() will also throw an exception when it should be throwing a rejection
* callers of asyncFunc() are expecting a rejection

* Solution 1:
    - we can wrap the whole body of asyncFunc in a try-catch statement and return a rejected Promise if an exception is thrown

In [ ]:
// Solution 1

function asyncFunc() {
    try {
        doSomethingSync();
        return doSomethingAsync()
            .then(result => {
            // ...
        });
    }
    catch(err) {
        return Promise.reject(err);
    }
}

* Solution 2:
    - we know that .then() will convert exceptions to rejections so we can just call doSomethingSync() in the .then() callback
* we can do this by creating a Promise chain using Promise.resolve() to get us into the .then() callback
    - the fulfillment value, which is undefined, does not matter for Promise.resolve()

In [1]:
// Solution 2

function asyncFunc() {
    return Promise.resolve()
        .then(() => {
            doSomethingSync();
            return doSomethingAsync();
        })
        .then(result => {
            // ...
        });
}

* Solution 3:
    - new Promise() constructor can also convert exceptions to rejections

In [2]:
// Solution 3

function asyncFunc() {
    return new Promise((resolve, reject) => {
        doSomethingSync();
        resolve(doSomethingAsync());
    })
    .then(result => {
        // ...
    })
}

## Promise-based functions start synchronously, settle asynchronously

* Promise-based functions are executed as follows:
    - their execution starts right away, synchronously (in the current task)
    - but the Promise they return is guaranteed to be settled asynchronously (in a later task) - if ever
* in the example, we can see that the callback of new Promise() is executed before the end of the code, while the result is delivered later (line A)
* Benefits of this approach:
    - starting synchronously helps avoid race conditions b/c we can rely on the order in which Promise-based functions begin
    - Chaining Promises won't starve other tasks of processing time b/c before a Promise is settled, there will always be a break, during which the event loop can run
    - Promise-based functions always return results asynchronously; we can be sure that there is never a synchronous return

In [4]:
function asyncFunc() {
    console.log('asyncFunc');
    return new Promise(
        (resolve, _reject) => {
            // this callback is called synchronously
            console.log('new Promise()');
            
            // but resolving the callback happens asynchronously
            resolve();
        });
}

console.log('START');
asyncFunc()
    .then(() => {
        // we need to wait for the new Promise callback to be resolved first
        // before this is console logged
        console.log('.then()'); // (A)
    });
console.log('END')



START
asyncFunc
new Promise()
END
.then()


## Promise combinator functions: working with Arrays of Promises

### What is a Promise combinator function?

* combinator is a pattern in functional programming for building structures using 2 kinds of funcitons:
    - primitive functions (primitives): creates atomic pieces
    - combinator functions (combinators): combine atomic and/or compound pieces to create compound pieces
* for JavaScript Promises:
    - primitives: Promise.resolve(), Promise.reject()
    - combinators: Promise.all(), Promise.race(), Promise.any(), Promise.allSettled()
        * for each of these combinators, the input is an iterable object containing 0+ Promises
            - so like an array of Promises
        * the output is 1 Promise

### Promise.all()

* returns a Promise which is:
    - Fulfilled if all promises are fulfilled
        * then its fulfillment value is an Array with the fulfillment values of promises
    - Rejected if at least one Promise is rejected
        * its rejection value is the rejection value of that Promise

In [1]:
// all promises are fulfilled so the array contains all fulfilled values
var promises = [
    Promise.resolve('result a'),
    Promise.resolve('result b'),
    Promise.resolve('result c')
];

Promise.all(promises)
    .then(arr => {
        arr.forEach(val => console.log(val))
    })

Promise { <pending> }

result a
result b
result c


In [3]:
// one promise is rejected
// so its rejection value is displayed in the .catch() callback
var promises = [
    Promise.resolve('result a'),
    Promise.resolve('result b'),
    Promise.reject('ERROR')
];

Promise.all(promises)
    .catch(err => {
        console.error(err)
    })

Promise { <pending> }

ERROR


#### Asynchronous .map() via Promise.all()

* higher order methods like .map() and .filter() are made for synchronous computations
* but if we passed in a Promise-based function into them, they would return an Array of Promises
    - normal code can't work with them
* but using Promise.all(), we can convert that Array of Promises into an array of normal values

In [4]:
function timesTwoAsync(x) {
    return new Promise(resolve => resolve(x * 2));
}

var arr = [1, 2, 3];
var promiseArr = arr.map(timesTwoAsync);
Promise.all(promiseArr)
    .then(result => {
        result.forEach(val => console.log(val));
    })

Promise { <pending> }

2
4
6


#### A more realistic .map() example

* we'll use .map() and Promise.all() to download text files from the web
* downloadText() uses the Promise-based fetch API to download a text file as a string
    - first it asynchronously retrieves a response (line A)
    - response.ok (line B) checks if there were errors such as "file not found"
    - if there weren't any, we use .text() (line C) to retrieve the contents of the file as a string

In [ ]:
function downloadText(url) {
    return fetch(url)
        .then(response => { // A
            if (!response.ok) { // B
                throw new Error(response.statusText);
            }
            return response.text(); // C
    });
}

var urls = [
    'http://example.com/first.txt',
    'http://example.com/second.txt',    
];

var promises = urls.map(url => downloadText(url));


// should print out ['First!', 'Second!']
Promise.all(promises)
    .then(arr => {
        arr.forEach(val => console.log(val))
})

#### A simple implementation of Promise.all()

* line A and line B are where the result Promise is settled
* after one of them is settled, the other can't change the settlement value b/c a Promise can only be settled once

In [7]:
// performs no safety checks

function all(iterable) {
    return new Promise((resolve, reject) => {
        let elementCount = 0;
        let result;
        
        let index = 0;
        for (const promise of iterable) {
            // Preserve the current value of the index
            const currentIndex = index;
            
            // Promise.then() takes in 2 arguments
            // a callback for the fulfilled case
            // and a callback for the rejected case
            promise.then(
                (value) => {
                    result[currentIndex] = value;
                    elementCount++;
                    // since we initialize result after all this code
                    // result.length === total number of items in the iterable
                    if (elementCount === result.length) {
                        resolve(result); // A
                    }
                },
                (err) => {
                    reject(err); // B
                }
            );
            index++;
        }
        
        // if the iterable is empty
        // just return an empty array
        if (index === 0) {
            resolve([]);
            return;
        }
        // Now we know how many Promises there are in 'iterable'.
        // we can wait until now with initializing result b/c
        // the callbacks of .then() are executed asynchronously
        result = new Array(index);
    });
}

### Promise.race()

* returns a Promise, q, which is settled as soon as the first Promise, p, among promises is settled
    - q has the same settlement value as p
* basically, whichever Promise is settled first among ALL of the promises is the one that decides what the settlement value is!

In [10]:
// settlement of the fulfilled Promise (line A) happens before the settlement of the rejected Promise (line B)
// so the result is also fulfilled (line C)

var promises = [
    new Promise((resolve, reject) => 
        setTimeout(() => resolve('result'), 100)), // A
    new Promise((resolve, reject) => 
        setTimeout(() => reject('ERROR'), 200)) // B
]

Promise.race(promises)
    .then(result => {
        console.log(result);
    })

Promise { <pending> }

result


In [14]:
// the rejection happens first
var assert = require('assert');

var promises = [
    new Promise((resolve, reject) => 
        setTimeout(() => resolve('result'), 200)), // A
    new Promise((resolve, reject) => 
        setTimeout(() => reject('ERROR'), 100)) // B
]

Promise.race(promises)
    .then(
        (result) => assert.fail(),
        (err) => console.error({err})
    )

Promise { <pending> }

{ err: 'ERROR' }


#### Using Promise.race() to time out a Promise

* __the main - relatively rare - use case for .race() is to time out Promises__
* what does "timing out a Promise" mean?:
    - if the input Promise is settled quickly enough, its settlement is passed on to the output Promise
        * i.e. whatever value comes out of the Promise you pass to timeout() will be returned from the single Promise created using Promise.race()
    - if it isn't settled quickly enough, the output Promise is rejected
* timing out prevents the input Promise from affecting the output since a Promise can only be settled once but it does not stop the asynchronous operation that produced the input Promise
    - i.e. the async code for the input Promise is still running even though the Promise returned by Promise.race() has already settled
* timeout() returns a Promise with the same settlement value as the one that settles first between the inputs:
    - input promise
    - promise that is rejected after timeoutInMs
* to produce the second Promise, timeout() uses the fact that resolving a pending Promise with a rejected Promise leads to the former being rejected

In [9]:
// returns a Promise that is resolved with value after ms milliseconds
function resolveAfter(ms, value = undefined) {
    return new Promise((resolve, reject) => {
        setTimeout(() => resolve(value), ms);
    })
}

function timeout(timeoutInMs, promise) {
    return Promise.race([
        promise,
        resolveAfter(timeoutInMs,
            Promise.reject(new Error('Operation timed out')),
        )
    ]);
}

In [11]:
// input Promise is fulfilled before the timeout happens
// node.js is being weird. this should only display the result and not the error

timeout(200, resolveAfter(100, 'Result!'))
    .then(result => {
        console.log({result})
    })

Promise { <pending> }

Error: Operation timed out
    at timeout (evalmachine.<anonymous>:12:28)
    at evalmachine.<anonymous>:4:1
    at Script.runInThisContext (node:vm:129:12)
    at Object.runInThisContext (node:vm:305:38)
    at run ([eval]:1020:15)
    at onRunRequest ([eval]:864:18)
    at onMessage ([eval]:828:13)
    at process.emit (node:events:526:28)
    at emit (node:internal/child_process:938:14)
    at processTicksAndRejections (node:internal/process/task_queues:84:21)

{ result: 'Result!' }


(node:12220) PromiseRejectionHandledWarning: Promise rejection was handled asynchronously (rejection id: 7)


In [8]:
// timeout happens before the input Promise is fulfilled

timeout(100, resolveAfter(2000, 'Result!'))
    .catch(err => {
        console.error({err})
    })

Promise { <pending> }

Error: Operation timed out
    at timeout (evalmachine.<anonymous>:12:28)
    at evalmachine.<anonymous>:3:1
    at Script.runInThisContext (node:vm:129:12)
    at Object.runInThisContext (node:vm:305:38)
    at run ([eval]:1020:15)
    at onRunRequest ([eval]:864:18)
    at onMessage ([eval]:828:13)
    at process.emit (node:events:526:28)
    at emit (node:internal/child_process:938:14)
    at processTicksAndRejections (node:internal/process/task_queues:84:21){
  err: Error: Operation timed out
      at timeout (evalmachine.<anonymous>:12:28)
      at evalmachine.<anonymous>:3:1
      at Script.runInThisContext (node:vm:129:12)
      at Object.runInThisContext (node:vm:305:38)
      at run ([eval]:1020:15)
      at onRunRequest ([eval]:864:18)
      at onMessage ([eval]:828:13)
      at process.emit (node:events:526:28)
      at emit (node:internal/child_process:938:14)
      at processTicksAndRejections (node:internal/process/task_queues:84:21)
}
(node:12220) PromiseRejectionHandled

#### A simple implementation of Promise.race()

* the result Promise is settled in either line A or line B
* once a Promise is settled, its settlement value can't be changed anymore

In [12]:
// performs no safety checks
function race(iterable) {
    return new Promise((resolve, reject) => {
        for (const promise of iterable) {
            promise.then(
                (value) => {
                    resolve(value); // A
                },
                (err) => {
                    reject(err); // B
                }
            );
        }
    });
}

### Promise.any() and AggregateError

* Promise.any() returns a Promise p
* how Promise.any() is settled depends on what happens in the iterable of Promises
    - the first promise fulfilled = p is resolved with it
    - if all promises rejected = p is rejected with an instance of AggregateError with all rejection values

In [ ]:
// type signature of AggregateError

class AggregateError extends Error {
    // Instance properties (complementing the ones of Error)
    errors: Array<any>;
    
    constructor(
        errors: Iterable<any>,
        message: string = '',
        options?: ErrorOptions // ES2022
    );
}

interface ErrorOptions {
    cause?: any; // ES2022
}

#### Two first examples

In [13]:
// if one Promise is fulfilled

var promises = [
    Promise.reject('ERROR A'),
    Promise.reject('ERROR B'),
    Promise.resolve('result')
];

Promise.any(promises)
    .then(result => {
        console.log({result})
    })

Promise { <pending> }

{ result: 'result' }


In [21]:
// if all Promises are rejected

var promises = [
    Promise.reject('ERROR A'),
    Promise.reject('ERROR B'),
    Promise.reject('ERROR C')
];

Promise.any(promises)
    .catch(aggregateError => {
        console.error({err: aggregateError.errors})
    })

Promise { <pending> }

{ err: [ 'ERROR A', 'ERROR B', 'ERROR C' ] }


#### Promise.any() vs Promise.all()

* they are inverses of each other
    - Promise.all(): the first rejection it finds is the one that will reject the result Promise, p, or if all of them work, then it returns an array with all fulfillment values
        * basically, either returns an array of all working values or the first rejection it finds
    - Promise.any(): the first fulfillment fulfills the result Promise, p, or it returns an array with all rejection values inside an error object (AggregateError)
        * basically, either returns an array of all rejection values or the first fulfilled value it finds
* they have different focuses:
    - Promise.all(): interested in _all_ fulfillments
        * one rejection = entire thing is rejected! __ALL OR NONE!__
    - Promise.any(): interested in the first fulfillment
        * if one of them works, then the entire thing works

#### Promise.any() vs Promise.race()

* Promise.race(): interested in settlements
    - the first Promise to settle (either rejection or fulfillment), wins the race
    - objective: which one of these Promises terminates first
* Promise.any(): interested in fulfillments
    - the first Promise to be fulfilled first, wins!
    - objective: which one of these Promises fulfills (succeeds) first
* __remember, a Promise can be in 3 states: pending, fulfilled, or rejected__
    - __a Promise is settled if it is either fulfilled or rejected__

#### Use cases for Promise.any()

* use Promise.any() if we have multiple async computations but only care about the first one that is successful
    - basically, whichever one succeeds first wins!

In [ ]:
// downloading resources

var resource = await Promise.any([
    fetch('http://example.com/first.txt')
        .then(response => response.text()),
    fetch('http://example.com/second.txt')
        .then(response => response.text())
]);

// same pattern allows us to use whichever module downloads quicker
var lodash = await Promise.any([
    import('https://primary.example.com/lodash'),
    import('https://secondary.example.com/lodash')
])

### Promise.allSettled()

* returns a Promise for an Array whose elements have the following type signature

In [ ]:
Promise.allSettled<T>(promises: Iterable<Promise<T>>)
    : Promise<Array<SettlementObject<T>>>

type SettlementObject<T> = FulfillmentObject<T> | RejectionObject;

interface FulfillmentObject<T> {
    status: 'fulfilled';
    value: T;
}

interface RejectionObject {
    status: 'rejected';
    reason: unknown;
}

* Promise.allSettled() returns Promise, out
* once all promises are settled, out is fulfilled with an Array where each element, e, corresponds to one promise, p, of promises
* if p is fulfilled with the fulfillment value, v, then e is:
    - { status: 'fulfilled', value: v }
* if p is rejected with the rejection value r, then e is:
    - { status: 'rejected', reason: r }
* BASICALLY:
    - Promise.allSettled() returns a Promise whose fulfillment value is an Array
    - this array contains SettlementObjects that correspond to the settlement value of each promise in the iterable
* unless there is an error with iterating over promises (iterable), then the ouput Promise, out, is NEVER rejected

#### A first demo of Promise.allSettled()

In [2]:
Promise.allSettled([
    Promise.resolve('a'),
    Promise.reject('b')
])
    .then(arr => {
        console.log({arr})
    })

Promise { <pending> }

{
  arr: [
    { status: 'fulfilled', value: 'a' },
    { status: 'rejected', reason: 'b' }
  ]
}


#### A longer example for Promise.allSettled()

* for this example, we are downloading multiple text files whose URLs are stored in an Array
* but we don't stop when there is an error, we want to keep going, which Promise.allSettled will allow us to do

In [ ]:
var urls = [
    'http://example.com/exists.txt',
    'http://example.com/missing.txt'
];

function downloadText(url) {
    return fetch(url)
        .then(response => { // A
            if (!response.ok) { // B
                throw new Error(response.statusText);
            }
            return response.text(); // C
    });
}

var result = Promise.allSettled(urls.map((u) => downloadText(u)));

result.then(
    arr => assert.deepEqual(
        arr,
        [
            {
                status: 'fulfilled',
                value: 'Hello!',
            },
            {
                status: 'rejected',
                reason: new Error('Not Found')
            }
        ]
    )
)

#### A simple implementation of Promise.allSettled()

In [4]:
// performs no safety checks

// notice that allSettled does not use reject
// unless there is a problem with iterating through the iterable, the promise is not rejected
function allSettled(iterable) {
    return new Promise((resolve, reject) => {
        let elementCount = 0;
        let result;
        
        function addElementToResult(i, elem) {
            result[i] = elem;
            elementCount++;
            if (elementCount === result.length) {
                resolve(result);
            }
        }
        
        let index = 0;
        for (const promise of iterable) {
            // capture current value of index
            const currentIndex = index;
            
            // this happens asynchronously
            // initializing result afterwards is fine
            
            // remember that .then() has 2 arguments
            // one for fulfillment and one for rejection
            promise.then(
                (value) => addElementToResult(
                    currentIndex,
                    {
                        status: 'fulfilled',
                        value
                    }
                ),
                (reason) => addElementToResult(
                    currentIndex,
                    {
                        status: 'rejected',
                        reason
                    }
                )
            )
            
            index++;
        }
        
        // if there are no promises in the iterable
        // just return an empty Array
        if (index === 0) {
            resolve([]);
            return;
        }
        
        // now we know how many promises there are in 'iterable'
        // we can wait until now with initializing 'redsult' b/c
        // the callbacks of .then() are executed asynchronously
        result = new Array(index);
    })
}

### Short-circuiting (advanced)

* short-circuiting = output Promise is __settled early__ before all input Promises in the iterable are settled
* these combinators short-circuit:
    - Promise.all(): output Promise is rejected as soon as one of the input Promises is rejected
        * all or none
    - Promise.race(): the output Promise is settled as soon as one input Promise is settled
        * it only cares about the fastest one
    - Promise.any(): output Promise is fulfilled as soon as one input Promise is fulfilled

## Concurrency and Promise.all() (advanced)

### Sequential execution vs. concurrent execution

* using .then() in the example below will execute Promise-based functions __sequentially__: only after asyncFunc1 has settled will we execute asyncFunc2

In [5]:
const asyncFunc1 = () => Promise.resolve('one');
const asyncFunc2 = () => Promise.resolve('two');

asyncFunc1()
    .then(result1 => {
        console.log({result1});
        return asyncFunc2();
    })
    .then(result2 => {
        console.log({result2});
    })

Promise { <pending> }

{ result1: 'one' }
{ result2: 'two' }


* Promise.all() helps executed Promise-based functions more concurrently

In [7]:
Promise.all([asyncFunc1(), asyncFunc2()])
    .then(arr => {
        console.log({arr})
    })

Promise { <pending> }

{ arr: [ 'one', 'two' ] }


### Concurrency tip: focus on when operations start

* focus on when asynchronous operations start, not on how their Promises are handled

In [ ]:
// asyncFunc1() and asyncFunc2() are executed concurrently b/c they are started at nearly the same time

function concurrentAll() {
    return Promise.all([asyncFunc1(), asyncFunc2()]);
}

function concurrentThen() {
    const p1 = asyncFunc1()
    const p2 = asyncFunc2();
    return p1.then(r1 => p2.then(r2 => [r1, r2]));
}

In [13]:
// asyncFunc1() and asyncFunc2() are executed sequentially
// asyncFunc2() is only invoked after the Promise of asyncFunc1() is fulfilled

function sequentialThen() {
    // different from concurrentThen b/c we did not INVOKE asyncFunc1() and asyncFunc2()
    // at nearly the same time
    
    // we see here that after asyncFunc1() is fulfilled and enters the .then() callback
    // that is when we invoke asyncFunc2()
    
    // if we look at concurrentThen, we see that both asyncFunc1() and asyncFunc2() are invoked right after the other
    // not in each others' .then() callbacks
    return asyncFunc1()
        .then(r1 => asyncFunc2()
            .then(r2 => [r1, r2]));
}

function sequentialAll() {
    // similar to sequentialThen
    // we invoke asyncFunc2() after asyncFunc1() has already been fulfilled and moved into the .then() callback
    const p1 = asyncFunc1();
    const p2 = p1.then(() => asyncFunc2());
    
    return Promise.all([p1, p2]);
}

### Promise.all() is fork-join

* Promise.all() is loosely related to the concurrency pattern "fork join"
* Fork: in line A, we are forking 2 asynchronous computations and executing them concurrently
* Join: in line B, we are joining these computations into a single "thread" which is started once all of them are done

In [ ]:
Promise.all([
    // (A) fork
    downloadText('http://example.com/first.txt'),
    downloadText('http://example.com/second.txt')
])
    .then(
        // (B) join
        (arr) => {
            assert.deepEqual(arr, ['First!', 'Second!']);
        }
    )

## Tips for chaining Promises

### Chaining mistake: losing the tail

* computation starts with the Promise returned by asyncFunc()
* but afterward, computation continues and another Promise is created via .then()
* .foo() returns the former Promise, but should return the latter
* basically, .then() is the Promise you want to return, not the one creating by the original asyncFunc() which is outdated

In [ ]:
// Problem:

// don't do this:
function foo() {
    const promise = asyncFunc();
    promise.then(result => {
        // ...
    });
    
    return promise;
}

// fix:
function foo() {
    const promise = asyncFunc();
    return promise.then(result => {
        // ...
    });
}

### Chaining mistake: nesting

* the .then() in line A is nested
* a flat structure would be better

In [ ]:
// problem:

// don't do this:
asyncFunc1()
    .then(result => {
        return asyncFunc2()
            .then(result2 => { // A
                // ...
            });
    })

// fix:
asyncFunc1()
    .then(result => {
        return asyncFunc2();
    })
    .then(result2 => {
        // ...
    });

### Chaining mistake: more nesting than necessary

In [ ]:
// avoidable nesting

// don't do this:
asyncFunc1()
    .then(result1 => {
        if (result1 < 0) {
            return asyncFuncA()
                .then(resultA => 'Result: ' + resultA);
        }
        else {
            return asyncFuncB()
                .then(resultB => 'Result: ' + resultB);
        }
    })

// fix:
asyncFunc1()
    .then(result1 => {
        return result1 < 0 ? asyncFuncA() : asyncFuncB();
    })
    .then(resultAB => {
        return 'Result: ' + resultAB;
    })

### Not all nesting is bad

* A: receiving an asynchronous result
* B: nesting so that we have access to variable connection inside the callback
* C: nested so we still have access to variable connection
***
* essentially, nesting is useful when we want to make use of the same variable in the promise methods

In [ ]:
// benefit from nesting:

db.open()
    .then(connection => { // A
        return connection.select({ name: 'Jane' })
            .then(result => { // B
                // Process result
                // Use 'connection' to make more queries
            })
            // ...
            .finally(() => {
                connection.close(); // C
            })
    })

### Chaining mistake: creating Promises instead of chaining

* in line A, we are creating a Promise to deliver the result of db.insert() which is unneccessary
* we don't need to create a Promise here
    - just needed to return the result of the .then() call
    - another benefit is that we don't need to catch and re-reject the failure of db.insert()
    - we can simply pass its rejection on to the caller of .insertInto()

In [ ]:
// problem:

// don't do this
class Model {
    insertInto(db) {
        return new Promise((resolve, reject) => { // A
            db.insert(this.fields)
                .then(resultCode => {
                    this.notifyObservers({event: 'created', model: this});
                    resolve(resultCode);
                })
                .catch(err => {
                    reject(err);
                })
        });
    }
    // ...
}

// fix:
class Model {
    insertInto(db) {
        return db.insert(this.fields)
            .then(resultCode => {
                this.notifyObservers({event: 'created', model: this});
                return resultCode;
            });
    }
    // ...
}

## Quick reference: Promise combinator functions

* Short-circuiting: the output Promise can be settled early (before every input Promise is settled)

### Promise.all()

* Fulfillment of p: if all input Promises are fulfilled
    - Value: array with the fulfillment values of the input Promises
* Rejection of p: if one input Promise is rejected
    - Value: rejection value of the input Promise
* Short-circuits: yes
* Use case: processing Arrays with Promises (rejections terminate processing)

In [ ]:
Promise.all<T>(promises: Iterable<Promise<T>>)
    : Promise<Array<T>>

### Promise.race()

* Settlement of p: if the first input Promise is settled
    - Value: settlement value of the input Promise
*  Short-circuits: yes
* Use case: reacting to the first settlement among multiple Promises

In [ ]:
Promise.race<T>(promises: Iterable<Promise<T>>)
    : Promise<T>

### Promise.any()

* Fulfillment of p: if one input Promise is fulfilled
    - Value: fulfillment value of the input Promise
* Rejection of p: if all input Promises are rejected
    - Value: AggregateError that contains the rejection values of the input Promises
* Short-circuits: yes
* Use case: among several asynchronous computations, we are only interested in the first successful one. that is, we are trying several approaches and the fastest one should win

In [ ]:
Promise.any<T>(promises: Iterable<Promise<T>>): Promise<T>
    
class AggregateError {
    constructor(errors: Iterable<any>, message: string);
    get errors(): Array<any>;
    get message(): string;
}

### Promise.allSettled()

* Fulfillment of p: if all input Promise are settled
    - Value: Array with one settlement object for each input Promise. A settlement object contains the kind of settlement and settlement value
* Rejection of p: if there is an error when iterating over the input Promises
* Short-circuits: no
* Use case: processing Arrays with Promises (rejections don't terminate processing)

In [ ]:
type SettlementObject<T> = FulfillmentObject<T> | RejectionObject;

interface FulfillmentObject<T> {
    status: 'fulfilled';
    value: T;
}

interface RejectionObject {
    status: 'rejected';
    reason: unknown;
}